In [25]:
import pandas as pd
import joblib

from Functions.original.utils.categorical import load_variable_sizes_from_metadata
import config.config as cc
from Functions.original.methods.general.generator import Generator
import torch.nn as nn
import torch

from sampler import sample

In [13]:
cc.continuous_activation = nn.LeakyReLU

In [29]:
model = Generator(
    60,
    load_variable_sizes_from_metadata(cc.metadata),
    hidden_sizes=[100,100,100],
    bn_decay=0.9
)

In [30]:
model.load_state_dict(torch.load('./data/generators/generator_20230521.pt', map_location=lambda storage, loc: storage))

<All keys matched successfully>

In [ ]:
train = pd.read_pickle('./data/common_dataprep/train_with_ei.pickle')
test = pd.read_pickle('./data/common_dataprep/test_with_ei.pickle')

In [ ]:
def make_glms(train, test, base_formula = ''):
    # No Input
    formula0 = "ClaimNb ~ 1"
    formula1 = f"ClaimNb ~ {base_formula}  VehPowerGLM + C(VehAgeGLM, Treatment(reference=2)) + C(DrivAgeGLM, Treatment(reference=5)) + BonusMalusGLM + VehGas + DensityGLM + C(Region, Treatment(reference='R24'))"
    formula2 = f"{formula1} + VehBrand"
    formula3 = f"{formula2} + AreaGLM"

    formulas = [formula0, formula1, formula2, formula3]
    devs = []
    
    for formulaa in formulas:
        glm0 = smf.glm(formula=formulaa, data=train, family=sm.families.Poisson(link=sm.families.links.log()), offset=np.log(train['Exposure'])).fit()
        devs += [metrics.poisson_deviance(glm0.predict(test, offset=np.log(test['Exposure'])), test['ClaimNb'])]
    
    assert min(devs) != devs[0], f'Issue with metrics: {dev}; random model predicts best?!'
    return min(devs)
    

In [ ]:
print(
    
    make_glms(train, test),
    make_glms(train, test, base_formula = 'EI_Density + EI_DrivAge + EI_BonusMalus1 + EI_BonusMalus2 + EI_VehAge +'),
    make_glms(train, test, base_formula = 'GDV_Area + GDV_VehAge + GDV_DrivAge + '),
    make_glms(train, test, base_formula = 'GDV_Area + GDV_VehAge + GDV_DrivAge + EI_Density + EI_DrivAge + EI_BonusMalus1 + EI_BonusMalus2 + EI_VehAge +')
)

In [ ]:
train_gan = sample(
    generator,
    cc.num_samples,
    cc.num_features,
    batch_size=500000,
    noise_size=60
)
train_gan = pd.DataFrame(train_gan)


In [ ]:
import joblib
specific = joblib.load('specific_transformer.pkl')

In [ ]:
train1 = specific.inverse_transform(train1, verbose=True)


array(['VehPower', 'VehAge', 'DrivAge', 'Density', 'BonusMalus',
       'Exposure', 'EI_Density', 'EI_DrivAge', 'EI_BonusMalus1',
       'EI_BonusMalus2', 'EI_VehAge', 'GDV_Area', 'GDV_VehAge',
       'GDV_DrivAge', 'VehBrand_B1', 'VehBrand_B10', 'VehBrand_B11',
       'VehBrand_B12', 'VehBrand_B13', 'VehBrand_B14', 'VehBrand_B2',
       'VehBrand_B3', 'VehBrand_B4', 'VehBrand_B5', 'VehBrand_B6',
       'VehGas_Diesel', 'VehGas_Regular', 'Region_R11', 'Region_R21',
       'Region_R22', 'Region_R23', 'Region_R24', 'Region_R25',
       'Region_R26', 'Region_R31', 'Region_R41', 'Region_R42',
       'Region_R43', 'Region_R52', 'Region_R53', 'Region_R54',
       'Region_R72', 'Region_R73', 'Region_R74', 'Region_R82',
       'Region_R83', 'Region_R91', 'Region_R93', 'Region_R94', 'Area_1',
       'Area_2', 'Area_3', 'Area_4', 'Area_5', 'Area_6', 'ClaimNb_0.0',
       'ClaimNb_1.0', 'ClaimNb_2.0', 'ClaimNb_3.0', 'ClaimNb_4.0'],
      dtype=object)

In [37]:
train1 = specific.inverse_transform(train1, verbose=True)


NameError: name 'specific' is not defined

In [34]:
test

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,...,VehAgeGLM,DrivAgeGLM,EI_Density,EI_DrivAge,EI_BonusMalus1,EI_BonusMalus2,EI_VehAge,GDV_Area,GDV_VehAge,GDV_DrivAge
5,13.0,1,0.520000,E,6,2,38,50.0,B12,Regular,...,2,4,0.053795,0.048885,0.042394,59.079562,0.05,52.0,40.0,47.0
6,15.0,1,0.450000,E,6,2,38,50.0,B12,Regular,...,2,4,0.046554,0.042305,0.036687,59.079562,0.05,52.0,40.0,47.0
13,32.0,1,0.050000,D,4,0,27,90.0,B12,Regular,...,1,3,0.004913,0.005459,0.009198,59.079562,0.05,46.5,40.0,47.0
21,49.0,2,0.810000,E,7,0,73,50.0,B12,Regular,...,1,7,0.084386,0.073732,0.066037,59.079562,0.05,52.0,40.0,48.0
22,50.0,1,0.060000,E,7,0,73,50.0,B12,Regular,...,1,7,0.006251,0.005462,0.004892,59.079562,0.05,52.0,40.0,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677984,6114302.0,0,0.010000,C,10,5,37,50.0,B12,Regular,...,2,4,0.000971,0.000932,0.000815,59.079562,0.05,43.5,46.0,47.0
677989,6114307.0,0,0.008219,D,10,2,41,50.0,B12,Regular,...,2,4,0.000824,0.000797,0.000670,59.079562,0.05,46.5,40.0,47.0
677990,6114308.0,0,0.008219,A,11,0,64,50.0,B12,Diesel,...,1,6,0.000796,0.000749,0.000670,59.079562,0.05,38.5,40.0,37.0
677993,6114311.0,0,0.008219,E,15,4,54,50.0,B12,Regular,...,2,6,0.000856,0.000820,0.000670,59.079562,0.05,52.0,46.0,37.0


In [90]:
cols = ['ClaimNb', 'VehBrand', 'VehGas', 'Region', 'Area', 'VehPower', 'VehAge',
       'DrivAge', 'Density', 'BonusMalus', 'Exposure', 'DensityGLM',
       'BonusMalusGLM', 'AreaGLM', 'VehPowerGLM', 'VehAgeGLM', 'DrivAgeGLM']
joined = pd.concat([train_gan, train])[cols]

In [91]:
# No Input
formula0 = "ClaimNb ~ 1"
formula1 = f"ClaimNb ~   VehPowerGLM + C(VehAgeGLM, Treatment(reference=2)) + C(DrivAgeGLM, Treatment(reference=5)) + BonusMalusGLM + VehGas + DensityGLM + C(Region, Treatment(reference='R24'))"
formula2 = f"{formula1} + VehBrand"
formula3 = f"{formula2} + AreaGLM"



formulas = [formula0, formula1, formula2, formula3]
devs = []

for formulaa in formulas:
    glm0 = smf.glm(formula=formulaa, data=train_gan, family=sm.families.Poisson(link=sm.families.links.log()), offset=np.log(train_gan['Exposure'])).fit()
    devs += [metrics.poisson_deviance(glm0.predict(test, offset=np.log(test['Exposure'])), test['ClaimNb'])]
    print(devs)

[32.91632850254276]
[32.91632850254276, 59.74845410521347]
[32.91632850254276, 59.74845410521347, 59.87506494269026]


TypeError: predict requires that you use a DataFrame when predicting from a model
that was created using the formula api.

The original error message returned by patsy is:
Object with dtype category cannot perform the numpy op isnan

In [ ]:
Bootstrap(9, n_bootstraps=3, n_train=5, n_test=4, random_state=0)